## Working with forcing conditions

In [ ]:
from hydromt.log import setuplog
from hydromt_sfincs import SfincsModel

In [ ]:
# Initialize SfincsModel with the artifact data catalog which contains data for North Italy
sf = SfincsModel(
    data_libs=["artifact_data"],
    root="tmp_example",
    mode="w+",
    logger=setuplog("", log_level=20),
)

In [ ]:
sf.setup_grid(
    x0=268650,
    y0=5018550,
    dx=150.0,
    dy=150.0,
    nmax=272,
    mmax=425,
    rotation=0,
    epsg=32633,
)

In [ ]:
sf.setup_config(
    tref="20100201 000000",
    tstart="20100201 000000",
    tstop="20100210 000000",
)

In [ ]:
# 2d precip
sf.setup_precip_forcing_from_grid(
    precip="era5_hourly",
    aggregate=True,
)
sf.write_forcing()

In [ ]:
# 1d uniform precip
sf.setup_precip_forcing_from_grid(
    precip="era5_hourly",
    aggregate=True,
)

In [ ]:
sf.forcing["precip"].to_pandas().to_csv("precip.csv")
sf.forcing.pop("precip", None)  # reset
sf.setup_precip_forcing(
    timeseries="precip.csv",
)
sf.plot_forcing()

In [ ]:
from hydromt_sfincs import utils

df = utils.read_timeseries("sfincs_compound//sfincs.bzs", tref=sf.config["tref"])
gdf = utils.read_xy("sfincs_compound//sfincs.bnd", crs=sf.crs)

sf.forcing.pop("bzs", None)  # reset
sf.forcing.pop("precip", None)  # reset

# add timeseries and locations
sf.setup_waterlevel_forcing(
    timeseries=df,
    locations=gdf,
    merge=True,
)
sf.plot_forcing()

In [ ]:
# merge (overwrite) existing timeseries with different time resoltiuon
# and add offset
sf.setup_waterlevel_forcing(
    timeseries=df.iloc[::5, [0]],
    locations=gdf.iloc[[0]],
    offset="dtu10mdt",
    merge=True,
)
sf.plot_forcing()

In [ ]:
# update timeseries from csv
df.to_csv("waterlevel.csv")
sf.setup_waterlevel_forcing(
    timeseries="waterlevel.csv",
    merge=True,
)
sf.plot_forcing()

In [ ]:
# overwrite forcing from geodataset (netcdf file)
sf.setup_waterlevel_forcing(
    geodataset="gtsmv3_eu_era5",
    offset="dtu10mdt",
    merge=False,
)
sf.plot_forcing()

In [ ]:
sf.write_forcing()
sf.write_config()

In [ ]:
# note that index number cannot be saved in ascii timeseries format
# and are stored as attributes of the geojson file
sf1 = SfincsModel(sf.root, mode="r")  # read mode
sf1.read_forcing()
sf1.plot_forcing()